In [1]:
import numpy as np
import pygsti as pig
import matplotlib.pyplot as plt
import random
import time
import pandas
from scipy.linalg import expm, sqrtm
# from qutip import *

from pygsti.modelmembers.operations import LindbladErrorgen as _LinbladErrorgen
from pygsti.modelmembers.operations import EmbeddedErrorgen as _EmbeddedErrorgen
from pygsti.modelmembers.operations import ExpErrorgenOp as _ExpErrorgenOp
from pygsti.modelmembers.operations import ComposedOp as _ComposedOp
import  pygsti.modelmembers as _mdmb
import pygsti.processors as _proc

from pygsti.tools import pdftools as _pdftools

from pygsti.circuits.gstcircuits import create_lsgst_circuits

from pygsti.models.memberdict import OrderedMemberDict as _OrderedMemberDict
from pygsti.baseobjs.label import Label as _Label, CircuitLabel as _CircuitLabel
from pygsti.circuits import Circuit
import pygsti.circuits as _circ
import pygsti.models as _mdl
from pygsti.modelmembers import operations as _op

In [2]:
def random_circuit(depth, proc_spec):
    """
    works for any 1-qubit model pack and the XYZICnot 2-qubit pack
    """
    layers = []
    for i in range(depth):
        gate = random.choice(proc_spec.gate_names)
        if gate == '(idle)':
            pass
        elif gate == 'Gcnot':
            layers.append((gate, 0, 1))
        else:
            layers.append((gate, random.choice(proc_spec.qubit_labels)))
    return Circuit(layers, proc_spec.qubit_labels)

In [3]:
def vector_from_outcomes(outcomes, num_outcomes):
    vecout = np.zeros((num_outcomes))
    for key in outcomes.keys():
        vecout[int(key[0], 2)] = outcomes[key]
    return(vecout)

def matrix_from_jacob(jacob, num_outcomes):
    matout = np.zeros((num_outcomes, len(jacob['0'*int(np.log2(num_outcomes))])))
    for key in jacob.keys():
        matout[int(key[0], 2), :] = np.array(jacob[key])
    return matout

def tensor_from_hessian(hessian, hilbert_dims):
    """
    returns a 3d-array that when dotted into the state returns the jacobian 
    """
    num_params = len(hessian['0'*int(np.log2(hilbert_dims))])
    tensor_out = np.zeros((hilbert_dims, num_params, num_params))
    for key in hessian.keys():
        tensor_out[int(key[0], 2), :, :] = hessian[key]
    return tensor_out

In [4]:
def compare_models(true_model, model1, model2, circuit_list, name1='x', name2='y'):
    distribution1 = []
    distribution2 = []
    mx = 0
    for idx, circ in enumerate(circuit_list): 
        hilbert_dims = 2**circ.width
        true_outcomes = true_model.probabilities(circ)
        outcome_set1 = model1.probabilities(circ)
        outcome_set2 = model2.probabilities(circ)
        dist1 = _pdftools.tvd(true_outcomes, outcome_set1)
        dist2 = _pdftools.tvd(true_outcomes, outcome_set2)
        if dist1 > mx:
            mx = dist1
        if dist2 > mx: 
            mx = dist2
        vtrue = vector_from_outcomes(true_outcomes, hilbert_dims)
        v1 = vector_from_outcomes(outcome_set1, hilbert_dims)
        v2 = vector_from_outcomes(outcome_set2, hilbert_dims)
        distribution1.append(vtrue - v1)
        distribution2.append(vtrue - v2)
        plt.scatter(dist1, dist2)
    plt.xlabel(name1)
    plt.ylabel(name2)
    plt.plot((0, mx), (0, mx), c='black')
    plt.show()
    return (distribution1, distribution2)

In [5]:
def calculate_jacobs(circ_list, model):
    jacobs = {}
    for circ in circ_list:
        hilbert_dims = 2**circ.width
        jacobs[circ] = matrix_from_jacob(model.sim.dprobs(circ), hilbert_dims)
    return jacobs

In [6]:
from pygsti.processors import CliffordCompilationRules as CCR

def make_rb_param(noise_model, model_pack, length_powers=5, circuits_at_length=10):
    pspec = model_pack.processor_spec()
    
    depths = [2**i for i in range(length_powers)]
    
    compilations = {'absolute': CCR.create_standard(pspec, 'absolute', ('paulis', '1Qcliffords'), verbosity=0),            
                'paulieq': CCR.create_standard(pspec, 'paulieq', ('1Qcliffords', 'allcnots'), verbosity=0)}

    design = pig.protocols.DirectRBDesign(model_pack.processor_spec(), compilations, depths, circuits_at_length, qubit_labels=model_pack.processor_spec().qubit_labels, sampler='edgegrab', 
                                           samplerargs=[0.5], randomizeout=True,
                                           citerations=20)
    pig.io.write_empty_protocol_data(design, 'RB_Data', clobber_ok=True)
    pig.io.fill_in_empty_dataset_with_fake_data(noise_model, 'RB_Data/data/dataset.txt', num_samples=1000)
    data = pig.io.load_data_from_dir('RB_Data')
    protocol = pig.protocols.RB()
    results = protocol.run(data)
    return results.fits['full'].estimates['r'] 

In [7]:
def make_dirichlet_covar(counts, hilbert_dims):
    total_counts = sum([counts[key] for key in counts.keys()])    
    count_vec = np.ones(hilbert_dims)
    for key in counts.keys():
        count_vec[int(key[0], 2)] += counts[key]
    prefactor = 1/( (total_counts + hilbert_dims)**2 * (total_counts + hilbert_dims + 1) )
    meas_covar = prefactor*(
        (total_counts + hilbert_dims)*np.diag(count_vec) - np.outer(count_vec, count_vec)
    )
    return meas_covar

In [8]:
def make_multinom_covar(prob_vec):
    return np.eye(len(prob_vec)) - np.outer(prob_vec, prob_vec)

In [9]:
def model_update(model, prior_covar, circ, counts, jacob0, hessian, model_covar=None):
    """
    create posterior state and covar from given counts
    
    https://iopscience.iop.org/article/10.1088/1367-2630/11/2/023028/pdf#nj294505eqn20
    """ 
    total_counts = sum([counts[key] for key in counts.keys()])    
    hilbert_dims = 2**(circ.width)
    prior_state = model.to_vector()
    
    # 0) find the probabiility for the circuit outcome under the model and calculate Jacobian if needed
    p_model = np.ones(hilbert_dims)
    probs = model.probabilities(circ)
    for key in probs.keys():
        p_model[int(key[0], 2)] += total_counts*probs[key]
    p_model = (1/(total_counts+hilbert_dims))*p_model
    
    jacob = jacob0 + hessian@prior_state
    
    # 1) calculate your observation frequencies
    observation = np.ones(hilbert_dims)
    for key in counts.keys():
        observation[int(key[0], 2)] += counts[key]
    observation = (1/(total_counts+hilbert_dims))*observation
            
    # 2) calculate the covaraiance of the observation 
    # see equ (7) in ref
    
    meas_covar = (1/(total_counts+hilbert_dims))*make_multinom_covar(observation)
    
    Qmat = len(circ)*model_covar
    
    # 3) Kalman update equations
    kgain = prior_covar@jacob.T@np.linalg.pinv(jacob@(prior_covar+Qmat)@jacob.T + meas_covar, 1e-8)
    
    innovation = observation - p_model
    
    post_state = prior_state + kgain@innovation
    post_covar = prior_covar - kgain@jacob@prior_covar
    model.from_vector(post_state)
    return (model, (post_covar+post_covar.T)/2, innovation)

In [10]:
from scipy.linalg import block_diag

def batch_update(model, prior_covar, circ_list, data_set, jdict, hdict, model_covar=None):
    prior_state = model.to_vector()
    Kmat = np.zeros((len(model.to_vector()),0))
    total_innov = np.zeros((0))
    total_jac = np.zeros((0,len(model.to_vector())))
    for circ in circ_list:
        counts = data_set[circ].counts
        total_counts = sum([counts[key] for key in counts.keys()])    
        hilbert_dims = 2**(circ.width)
        prior_state = model.to_vector()

        # 0) find the probabiility for the circuit outcome under the model and calculate Jacobian if needed
        p_model = np.ones(hilbert_dims)
        probs = model.probabilities(circ)
        for key in probs.keys():
            p_model[int(key[0], 2)] += total_counts*probs[key]
        p_model = (1/(total_counts+hilbert_dims))*p_model

        jacob = jdict[circ] + hdict[circ]@prior_state

        # 1) calculate your observation frequencies
        observation = np.ones(hilbert_dims)
        for key in counts.keys():
            observation[int(key[0], 2)] += counts[key]
        observation = (1/(total_counts+hilbert_dims))*observation

        # 2) calculate the covaraiance of the observation 
        # see equ (7) in ref

        meas_covar = make_multinom_covar(observation)
        if model_covar is not None:
            meas_covar += model_covar

        # 3) Kalman update equations
        kgain = prior_covar@jacob.T@np.linalg.pinv(jacob@prior_covar@jacob.T + meas_covar, 1e-8)

        innovation = observation - p_model
        Kmat = np.hstack((Kmat, kgain))
        total_innov = np.hstack([total_innov, innovation])
        total_jac = np.vstack([total_jac, jacob])
    #endfor
        
    # make block matrices
    post_state = prior_state + Kmat@total_innov
    post_covar = prior_covar - Kmat@total_jac@prior_covar
    model.from_vector(post_state)
    return (model, (post_covar+post_covar.T)/2, total_innov)

In [11]:
from scipy.linalg import block_diag

class ExtendedKalmanFilter():
    def __init__(self, model, P0, hilbert_dims, window_size=20):
        self.model = model
        self.P = P0
        self.innovs = [np.ones(hilbert_dims)]
        self.window_size = window_size
        
    def update(self, circ_list, data_set, jdict, hdict, stab_noise=None, max_itr=1, itr_eps=1e-4):
        Smat = np.zeros((0,0))
        
        
        prior_covar = self.P
        prior_state = self.model.to_vector()
        
        for itr in range(max_itr):
            total_innov = np.zeros((0))
            total_jac = np.zeros((0,len(self.model.to_vector())))
        
            for circ in circ_list:
                counts = data_set[circ].counts
                total_counts = sum([counts[key] for key in counts.keys()])    
                if total_counts > num_samples:
                    print(total_counts)
                hilbert_dims = 2**(circ.width)
                prior_state = self.model.to_vector()

                # 0) find the mean estimate for the circuit outcome under the self.model
                p_model = np.ones(hilbert_dims)
                probs = self.model.probabilities(circ)
                for key in probs.keys():
                    p_model[int(key[0], 2)] += total_counts*probs[key]
                p_model = (1/(total_counts+hilbert_dims))*p_model

                # calculate jacobian
                jacob = jdict[circ] + hdict[circ]@prior_state

                # 1) calculate your observation frequencies
                observation = np.ones(hilbert_dims)
                for key in counts.keys():
                    observation[int(key[0], 2)] += counts[key]
                observation = (1/(total_counts+hilbert_dims))*observation

                # 2) calculate the covaraiance of the observation and add model noise
                meas_covar = (1/(total_counts+hilbert_dims))*make_multinom_covar(observation)
                if stab_noise is not None:
                    meas_covar += stab_noise
                    
                # 3) Kalman gain
                smat = np.linalg.pinv(jacob@prior_covar@jacob.T + meas_covar, 1e-8)
                
                innovation = observation - p_model

                Smat = block_diag(Smat, smat)
                total_innov = np.hstack([total_innov, innovation])
                total_jac = np.vstack([total_jac, jacob])
            Kgain = prior_covar@total_jac.T@Smat
            post_state = prior_state + Kgain@total_innov
            self.P = prior_covar - Kgain@total_jac@prior_covar
            if np.linalg.norm(post_state - prior_state) < itr_eps:
                break
        self.model.from_vector(post_state)

In [12]:
from pygsti.modelpacks import smq1Q_XYI as _smq1Q_XYI
from pygsti.modelpacks import smq1Q_XYZI as _smq1Q_XYZI
from pygsti.modelpacks import smq1Q_XYI as _smq1Q_XZ
from pygsti.modelpacks import smq2Q_XYZICNOT as _smq2Q_XYZICNOT

In [15]:
# make random circuits and design matrices
N_circs = 1000
depth = 15

random_circuits = []
for n in range(N_circs):
    random_circuits.append(random_circuit(random.choice(range(depth)), MODEL_PACK.processor_spec()))

jdict = dict()
hdict = dict()
for idx, circ in enumerate(random_circuits):
    if idx%10 == 0:
        print(idx/10, N_circs/10)
    jdict[circ] = matrix_from_jacob(ref_model.sim.dprobs(circ), 2**circ.width)
    hdict[circ] = tensor_from_hessian(ref_model.sim.hprobs(circ), 2**circ.width)

0.0 100.0
1.0 100.0
2.0 100.0
3.0 100.0
4.0 100.0
5.0 100.0
6.0 100.0
7.0 100.0
8.0 100.0
9.0 100.0
10.0 100.0
11.0 100.0
12.0 100.0
13.0 100.0
14.0 100.0
15.0 100.0
16.0 100.0
17.0 100.0
18.0 100.0
19.0 100.0
20.0 100.0
21.0 100.0
22.0 100.0
23.0 100.0
24.0 100.0
25.0 100.0
26.0 100.0
27.0 100.0
28.0 100.0
29.0 100.0
30.0 100.0
31.0 100.0
32.0 100.0
33.0 100.0
34.0 100.0
35.0 100.0
36.0 100.0
37.0 100.0
38.0 100.0
39.0 100.0
40.0 100.0
41.0 100.0
42.0 100.0
43.0 100.0
44.0 100.0
45.0 100.0
46.0 100.0
47.0 100.0
48.0 100.0
49.0 100.0
50.0 100.0
51.0 100.0
52.0 100.0
53.0 100.0
54.0 100.0
55.0 100.0
56.0 100.0
57.0 100.0
58.0 100.0
59.0 100.0
60.0 100.0
61.0 100.0
62.0 100.0
63.0 100.0
64.0 100.0
65.0 100.0
66.0 100.0
67.0 100.0
68.0 100.0
69.0 100.0
70.0 100.0
71.0 100.0
72.0 100.0
73.0 100.0
74.0 100.0
75.0 100.0
76.0 100.0
77.0 100.0
78.0 100.0
79.0 100.0
80.0 100.0
81.0 100.0
82.0 100.0
83.0 100.0
84.0 100.0
85.0 100.0
86.0 100.0
87.0 100.0
88.0 100.0
89.0 100.0
90.0 100.0
91.0 100.

# Extended Kalman Filter 

In [58]:
# make a model from a model pack
MODEL_PACK = _smq1Q_XYZI
ref_model = MODEL_PACK.target_model('H+S')
filter_model = MODEL_PACK.target_model('H+S')
noise_model = MODEL_PACK.target_model()
noise_model = noise_model.depolarize(max_op_noise=0.01)
noise_model = noise_model.rotate(max_rotate=0.01)

In [59]:
import pygsti
# sample circuits
num_samples = 5000
edesign = pig.protocols.ExperimentDesign(random_circuits)
def simulate_taking_data(data_template_filename, noise_model, num_samples):
    """Simulate taking 1-qubit data and filling the results into a template dataset.txt file"""
    datagen_model = noise_model
    pygsti.io.fill_in_empty_dataset_with_fake_data(datagen_model, data_template_filename, num_samples=num_samples, seed=1234)
    
pygsti.io.write_empty_protocol_data(edesign, 'test', clobber_ok=True)

# -- fill in the dataset file in tutorial_files/test_gst_dir/data/dataset.txt --
simulate_taking_data("test/data/dataset.txt", noise_model, num_samples)  # REPLACE with actual data-taking

data = pygsti.io.load_data_from_dir('test')
data_set = data.dataset

In [60]:
# make rb param
rb_param = make_rb_param(noise_model, MODEL_PACK)
print(rb_param, np.sqrt(rb_param))

- Sampling 10 circuits at DRB length 1 (1 of 5 depths) with seed 475444
- Sampling 10 circuits at DRB length 2 (2 of 5 depths) with seed 475454
- Sampling 10 circuits at DRB length 4 (3 of 5 depths) with seed 475464
- Sampling 10 circuits at DRB length 8 (4 of 5 depths) with seed 475474
- Sampling 10 circuits at DRB length 16 (5 of 5 depths) with seed 475484
0.0012088045433235328 0.034767866533963986


In [61]:
# Setup the Extended Kalman filter
%matplotlib

hilbert_dims = 2**len(MODEL_PACK.processor_spec().qubit_labels)
num_params = len(filter_model.to_vector())

xvec = np.zeros(num_params)
P = 2*np.sqrt(rb_param)*np.eye(num_params)

filter_model.from_vector(xvec)

kf = ExtendedKalmanFilter(filter_model, P, hilbert_dims, 50)

Using matplotlib backend: Qt5Agg


In [62]:
# run it
plotting = True
max_itr = 5
itr_epsilon = 1e-4 # minimum prior-posterior parameter distance to continue iterating

batch_size = 1


for idx in range(0, len(random_circuits), batch_size): 
    if idx+batch_size <= len(random_circuits):
        batch = random_circuits[idx:idx+batch_size]
    else:
        batch = random_circuits[idx::]
        
    if len(batch) > 0:
        if(plotting):
            filter_model = kf.model
            tot_naive_error = 0
            tot_filter_error = 0
            for circ in batch:
                naive_error = _pdftools.tvd(ref_model.probabilities(circ), data_set[circ].fractions)
                error = _pdftools.tvd(filter_model.probabilities(circ), data_set[circ].fractions)
                tot_naive_error += naive_error
                tot_filter_error += error
            tot_naive_error *= 1/len(batch)
            tot_filter_error *= 1/len(batch)
            plt.scatter(idx, tot_naive_error, c='blue', alpha=0.5)
            plt.scatter(idx, tot_filter_error, c='red', alpha=0.5)
            
        kf.update(batch, data_set, jdict, hdict, 0.01*np.eye(2))

        

In [63]:
# make random circuits 
plt.figure()
N_circs = 500
depth = 20
new_circ_list = []
for n in range(N_circs):
    new_circ_list.append(random_circuit(depth, MODEL_PACK.processor_spec()))

ref_innov, filter_innov = compare_models(noise_model, ref_model, filter_model, new_circ_list, 'ref', 'filter')

In [28]:
plt.figure()
plt.hist([p[0] for p in filter_innov], bins=100)
#plt.hist([p[0] for p in ref_innov])

(array([ 1.,  0.,  0.,  0.,  3.,  0.,  0.,  1.,  0.,  1.,  0.,  3.,  1.,
         1.,  1.,  3.,  3.,  1.,  5.,  1.,  4.,  6.,  3.,  3.,  6.,  6.,
         4.,  3.,  7.,  5.,  8.,  6.,  5., 12., 13., 12., 13., 10., 14.,
        14., 16., 12., 23., 15., 12., 12., 12., 15., 17., 11.,  9., 13.,
         9.,  4., 11., 14., 11.,  4., 10., 11.,  7., 10.,  7.,  6.,  6.,
         1.,  9.,  5.,  4.,  4.,  2.,  2.,  2.,  2.,  1.,  0.,  4.,  1.,
         1.,  1.,  0.,  2.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]),
 array([-4.57839712e-03, -4.47776411e-03, -4.37713111e-03, -4.27649810e-03,
        -4.17586509e-03, -4.07523209e-03, -3.97459908e-03, -3.87396607e-03,
        -3.77333307e-03, -3.67270006e-03, -3.57206705e-03, -3.47143405e-03,
        -3.37080104e-03, -3.27016803e-03, -3.16953502e-03, -3.06890202e-03,
        -2.96826901e-03, -2.86763600e-03, -2.76700300e-03, -2.66636999e-03,
        -2.56573698e-03, -2.46510398e-03, -2.36447097e

----

## Now test on GST circuits

In [20]:
gst_circuits = exp_design.all_circuits_needing_data

jdict_gst = dict()
hdict_gst = dict()
for idx, circ in enumerate(gst_circuits):
    if idx%10 == 0:
        print(idx/len(gst_circuits), " percent done")
    jdict_gst[circ] = matrix_from_jacob(ref_model.sim.dprobs(circ), 2**circ.width)
    hdict_gst[circ] = tensor_from_hessian(ref_model.sim.hprobs(circ), 2**circ.width)

0.0  percent done
0.008183306055646482  percent done
0.016366612111292964  percent done
0.024549918166939442  percent done
0.03273322422258593  percent done
0.04091653027823241  percent done
0.049099836333878884  percent done
0.057283142389525366  percent done
0.06546644844517185  percent done
0.07364975450081833  percent done
0.08183306055646482  percent done
0.09001636661211129  percent done
0.09819967266775777  percent done
0.10638297872340426  percent done
0.11456628477905073  percent done
0.12274959083469722  percent done
0.1309328968903437  percent done
0.13911620294599017  percent done
0.14729950900163666  percent done
0.15548281505728315  percent done
0.16366612111292964  percent done
0.1718494271685761  percent done
0.18003273322422259  percent done
0.18821603927986907  percent done
0.19639934533551553  percent done
0.20458265139116202  percent done
0.2127659574468085  percent done
0.220949263502455  percent done
0.22913256955810146  percent done
0.23731587561374795  percent d

In [49]:
# make a model from a model pack
ref_model = MODEL_PACK.target_model('H+S')
filter_model = MODEL_PACK.target_model('H+S')
noise_model = MODEL_PACK.target_model()
noise_model = noise_model.depolarize(max_op_noise=0.01, max_spam_noise=0.001)
noise_model = noise_model.rotate(max_rotate=0.01)

In [50]:
#Step 1: create an "experiment design" for doing GST on the std1Q_XYI gate set
target_model = MODEL_PACK.target_model('H+S')      # a Model object
prep_fiducials = MODEL_PACK.prep_fiducials()  # a list of Circuit objects
meas_fiducials = MODEL_PACK.meas_fiducials()  # a list of Circuit objects
germs = MODEL_PACK.germs()                    # a list of Circuit objects
maxLengths = [1,2,4,8,16]
exp_design = pygsti.protocols.StandardGSTDesign(target_model, prep_fiducials, meas_fiducials,
                                                germs, maxLengths)

In [64]:
num_samples=5000

def simulate_taking_data(data_template_filename):
    """Simulate taking 1-qubit data and filling the results into a template dataset.txt file"""
    datagen_model = noise_model
    pygsti.io.fill_in_empty_dataset_with_fake_data(datagen_model, data_template_filename, num_samples=1000, seed=1234)
    
pygsti.io.write_empty_protocol_data(exp_design, '../tutorial_files/test_gst_dir', clobber_ok=True)

# -- fill in the dataset file in tutorial_files/test_gst_dir/data/dataset.txt --
simulate_taking_data("../tutorial_files/test_gst_dir/data/dataset.txt")  # REPLACE with actual data-taking

data = pygsti.io.load_data_from_dir('../tutorial_files/test_gst_dir')
dataset_gst = data.dataset

In [ ]:
# make rb param
rb_param = make_rb_param(noise_model, MODEL_PACK)
print(rb_param, np.sqrt(rb_param))

In [65]:
# Setup the Extended Kalman filter
%matplotlib

hilbert_dims = 2**len(MODEL_PACK.processor_spec().qubit_labels)
num_params = len(filter_model.to_vector())

xvec = np.zeros(num_params)
P = 2*np.sqrt(rb_param)*np.eye(num_params)

#filter_model.from_vector(xvec)

kf = ExtendedKalmanFilter(filter_model, P, hilbert_dims, 50)

Using matplotlib backend: Qt5Agg


In [66]:
kf = ExtendedKalmanFilter(filter_model, P, hilbert_dims, 50)

In [67]:
# run it
plotting = True
if(plotting):
    plt.figure()
max_itr = 10
itr_epsilon = 1e-6 # minimum prior-posterior parameter distance to continue iterating

batch_size = 1

for idx in range(0, len(gst_circuits), batch_size): 
    if idx+batch_size <= len(gst_circuits):
        batch = gst_circuits[idx: idx+batch_size]
    else:
        batch = gst_circuits[idx*batch_size::]
        
    if len(batch) > 0:
        kf.update(batch, dataset_gst, jdict_gst, hdict_gst, 0.05*np.eye(2))

        if(plotting):
            filter_model = kf.model
            tot_naive_error = 0
            tot_filter_error = 0
            for circ in batch:
                naive_error = _pdftools.tvd(ref_model.probabilities(circ), dataset_gst[circ].fractions)
                error = _pdftools.tvd(filter_model.probabilities(circ), dataset_gst[circ].fractions)
                tot_naive_error += naive_error
                tot_filter_error += error
            tot_naive_error *= 1/len(batch)
            tot_filter_error *= 1/len(batch)
            plt.scatter(idx, tot_naive_error, c=Extended'blue', alpha=0.5)
            plt.scatter(idx, tot_filter_error, c='red', alpha=0.5)

In [68]:
filter_model = kf.model

In [75]:
# make random circuits 
plt.figure()
N_circs = 350
depth = 100
new_circ_list = []
for n in range(N_circs):
    new_circ_list.append(random_circuit(depth, MODEL_PACK.processor_spec()))

plt.figure()
ref_innov, filter_innov = compare_models(noise_model, ref_model, filter_model, new_circ_list, 'ref', 'filter')

In [70]:
#run the GST protocol and create a report 
gst_protocol = pygsti.protocols.StandardGST('full TP,CPTP,Target')
results = gst_protocol.run(data)

-- Std Practice:  Iter 1 of 3  (full TP) --: 
  --- Iterative GST: [##################################################] 100.0%  1222 circuits ---
  Iterative GST Total Time: 4.8s
-- Std Practice:  Iter 2 of 3  (CPTP) --: 


  --- Iterative GST: [##################################################] 100.0%  1222 circuits ---
  Iterative GST Total Time: 41.0s
-- Std Practice:  Iter 3 of 3  (Target) --: 


In [72]:
gst_model = results.estimates['CPTP'].models['final iteration estimate']

In [76]:
filter_innov, gst_innov = compare_models(noise_model, filter_model, gst_model, new_circ_list, 'filter', 'gst')

In [ ]:
gst_model.probabilities(random_circuits[0])